In [2]:
import sys
sys.path.append('../../Share/')
sys.path.append('../../V5 Self Supervised Learning')

import self_supervised_v1
import matplotlib.pyplot as plt
import baseline, config

import warnings
warnings.filterwarnings('ignore')

trainer_Minjeong = baseline.ModelTrainer(config, subject="Minjeong")
trainer_Carlson = baseline.ModelTrainer(config, subject="Carlson")
trainer_Harold = baseline.ModelTrainer(config, subject="Harold")
trainer_Hunmin = baseline.ModelTrainer(config, subject="Hunmin")
trainer_Brian = baseline.ModelTrainer(config, subject="Brian")
trainer_Xianyu = baseline.ModelTrainer(config, subject="Xianyu")

In [3]:
def evaluate_model(model, data, labels):
    return model.evaluate(data, labels, verbose=0)[1]

import pandas as pd
import numpy as np
from sklearn.utils import resample

def down_sample(X, y):
    X = np.array(X)
    y = np.array(y)

    # 클래스별 최소 개수
    unique_classes, counts = np.unique(y, return_counts=True)
    min_count = counts.min()

    X_balanced, y_balanced = [], []

    for cls in unique_classes:
        idx = np.where(y == cls)[0]  # 해당 클래스 인덱스
        down_idx = resample(idx, replace=False, n_samples=min_count, random_state=42)
        X_balanced.append(X[down_idx])
        y_balanced.append(y[down_idx])

    # 합치기
    X_balanced = np.concatenate(X_balanced, axis=0)
    y_balanced = np.concatenate(y_balanced, axis=0)

    return X_balanced, y_balanced


In [6]:
from tensorflow.keras.models import load_model

model = load_model('./model_K10_H2.h5')

In [7]:
Same_Session_Test_Acc, Next_Session_Test_Acc = [], []
Unseen_subject_acc_dict = {"H": [], "C": [], "B": [], "X": [], "M": []}
trainers_unseen = {
    "H": trainer_Hunmin,
    "C": trainer_Carlson,
    "B": trainer_Brian,
    "X": trainer_Xianyu,
    "M": trainer_Minjeong
}



In [9]:
import meta

trainer_sub = trainer_Carlson
final_session = len(config.Info_sub_C)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)

##### Few-shot iteration => 여기서 수정을 줘야함
for k in range(final_session-1):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=2                          # Reptile meta step
    )
    accuracies = Meta.train(X_train, y_train, X_test, y_test, meta.get_data_Meta, N_way=6, K_shot=20)
    acc = evaluate_model(model, X_TEST, y_TEST)
    print(acc, accuracies)

Returning K-th session data: Exp_2025-08-01-v2/E9AD0E7DCC2B/
Returning K-th session data: Exp_2025-06-30-v1/E9AD0E7DCC2B/


NotImplementedError: numpy() is only available when eager execution is enabled.

In [10]:
X_train.shape

(1764, 4, 14, 1)

In [ ]:
Unseen_subject_acc_dict

In [ ]:
SUBJECT = "Carlson"
Threshold_K = 10
trainer = baseline.ModelTrainer(config, subject=SUBJECT)
X_init, y_init, X_init_test, y_init_test = trainer.return_until_K_data(K=Threshold_K, train_ratio=0.8)
print(X_init.shape, y_init.shape, X_init_test.shape, y_init_test.shape)

model = self_supervised_v1.build_model()
result = model.fit(X_init, y_init, validation_data=(X_init_test, y_init_test), epochs=100, batch_size=64, verbose=0)
model.save('./model_K10_C.h5')

In [ ]:
Same_Session_Test_Acc, Next_Session_Test_Acc = [], []
#Unseen_subject_acc_dict = {"H": [], "C": [], "H2": [], "X": [], "M": []}
Unseen_subject_acc_dict = {"H": [], "B": [], "H2": [], "X": [], "M": []}
trainers_unseen = {
    "H": trainer_Hunmin,
    #"C": trainer_Carlson,
    "B": trainer_Brian,
    "H2": trainer_Harold,
    "X": trainer_Xianyu,
    "M": trainer_Minjeong
}

# Unseen subject 테스트
for key, trainer_sub in trainers_unseen.items():
    print(f"Start : {key}")
    if key == "H":
        final_session = len(config.Info_sub_H)
    #elif key == "C":
    #    final_session = len(config.Info_sub_C)
    elif key == "B":
        final_session = len(config.Info_sub_B)
    elif key == "H2":
        final_session = len(config.Info_sub_H2)
    elif key == "X":
        final_session = len(config.Info_sub_X)
    elif key == "M":
        final_session = len(config.Info_sub_M)

    unseen_X, unseen_y, _, _ = trainer_sub.return_until_K_data(K=final_session, train_ratio=0.8)
    X_test, y_test = down_sample(unseen_X, unseen_y)
    acc = evaluate_model(model, X_test, y_test)
    #print(pd.Series(y_test).value_counts())
    Unseen_subject_acc_dict[key].append(acc)

In [ ]:
Unseen_subject_acc_dict

In [ ]:
SUBJECT = "Xianyu"
Threshold_K = 10
trainer = baseline.ModelTrainer(config, subject=SUBJECT)
X_init, y_init, X_init_test, y_init_test = trainer.return_until_K_data(K=Threshold_K, train_ratio=0.8)
print(X_init.shape, y_init.shape, X_init_test.shape, y_init_test.shape)

model = self_supervised_v1.build_model()
result = model.fit(X_init, y_init, validation_data=(X_init_test, y_init_test), epochs=100, batch_size=64, verbose=0)
model.save('./model_K10_X.h5')

In [ ]:
Same_Session_Test_Acc, Next_Session_Test_Acc = [], []
Unseen_subject_acc_dict = {"H": [], "C": [], "B": [], "H2": [], "M": []}
#Unseen_subject_acc_dict = {"H": [], "B": [], "H2": [], "X": [], "M": []}
trainers_unseen = {
    "H": trainer_Hunmin,
    #"C": trainer_Carlson,
    "B": trainer_Brian,
    "H2": trainer_Harold,
    "X": trainer_Xianyu,
    "M": trainer_Minjeong
}

# Unseen subject 테스트
for key, trainer_sub in trainers_unseen.items():
    print(f"Start : {key}")
    if key == "H":
        final_session = len(config.Info_sub_H)
    elif key == "C":
        final_session = len(config.Info_sub_C)
    elif key == "B":
        final_session = len(config.Info_sub_B)
    elif key == "H2":
        final_session = len(config.Info_sub_H2)
    #elif key == "X":
    #    final_session = len(config.Info_sub_X)
    elif key == "M":
        final_session = len(config.Info_sub_M)

    unseen_X, unseen_y, _, _ = trainer_sub.return_until_K_data(K=final_session, train_ratio=0.8)
    X_test, y_test = down_sample(unseen_X, unseen_y)
    acc = evaluate_model(model, X_test, y_test)
    #print(pd.Series(y_test).value_counts())
    Unseen_subject_acc_dict[key].append(acc)

In [ ]:
Unseen_subject_acc_dict

In [ ]:
SUBJECT = "Minjeong"
Threshold_K = 10
trainer = baseline.ModelTrainer(config, subject=SUBJECT)
X_init, y_init, X_init_test, y_init_test = trainer.return_until_K_data(K=Threshold_K, train_ratio=0.8)
print(X_init.shape, y_init.shape, X_init_test.shape, y_init_test.shape)

model = self_supervised_v1.build_model()
result = model.fit(X_init, y_init, validation_data=(X_init_test, y_init_test), epochs=100, batch_size=64, verbose=0)
model.save('./model_K10_M.h5')

In [ ]:
Same_Session_Test_Acc, Next_Session_Test_Acc = [], []
Unseen_subject_acc_dict = {"H": [], "C": [], "B": [], "H2": [], "X": []}
#Unseen_subject_acc_dict = {"H": [], "B": [], "H2": [], "X": [], "M": []}
trainers_unseen = {
    "H": trainer_Hunmin,
    "C": trainer_Carlson,
    "B": trainer_Brian,
    "H2": trainer_Harold,
    "X": trainer_Xianyu,
    #"M": trainer_Minjeong
}

# Unseen subject 테스트
for key, trainer_sub in trainers_unseen.items():
    print(f"Start : {key}")
    if key == "H":
        final_session = len(config.Info_sub_H)
    elif key == "C":
        final_session = len(config.Info_sub_C)
    elif key == "B":
        final_session = len(config.Info_sub_B)
    elif key == "H2":
        final_session = len(config.Info_sub_H2)
    elif key == "X":
        final_session = len(config.Info_sub_X)
    #elif key == "M":
    #    final_session = len(config.Info_sub_M)

    unseen_X, unseen_y, _, _ = trainer_sub.return_until_K_data(K=final_session, train_ratio=0.8)
    X_test, y_test = down_sample(unseen_X, unseen_y)
    acc = evaluate_model(model, X_test, y_test)
    #print(pd.Series(y_test).value_counts())
    Unseen_subject_acc_dict[key].append(acc)

In [ ]:
Unseen_subject_acc_dict

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('C:\Users\hml76\PycharmProjects\MindForce\github\V5 Self Supervised Learning\V1 - Semi Supervised Learning in online(Pseudo-labeling)/model_K10_H.h5')

In [ ]:
Same_Session_Test_Acc, Next_Session_Test_Acc = [], []
Unseen_subject_acc_dict = {"B": [], "C": [], "H2": [], "X": [], "M": []}
#Unseen_subject_acc_dict = {"H": [], "B": [], "H2": [], "X": [], "M": []}
trainers_unseen = {
    #"H": trainer_Hunmin,
    "C": trainer_Carlson,
    "B": trainer_Brian,
    "H2": trainer_Harold,
    "X": trainer_Xianyu,
    "M": trainer_Minjeong
}

# Unseen subject 테스트
for key, trainer_sub in trainers_unseen.items():
    print(f"Start : {key}")
    #if key == "H":
    #    final_session = len(config.Info_sub_H)
    if key == "C":
        final_session = len(config.Info_sub_C)
    elif key == "B":
        final_session = len(config.Info_sub_B)
    elif key == "H2":
        final_session = len(config.Info_sub_H2)
    elif key == "X":
        final_session = len(config.Info_sub_X)
    elif key == "M":
        final_session = len(config.Info_sub_M)

    unseen_X, unseen_y, _, _ = trainer_sub.return_until_K_data(K=final_session, train_ratio=0.8)
    X_test, y_test = down_sample(unseen_X, unseen_y)
    acc = evaluate_model(model, X_test, y_test)
    #print(pd.Series(y_test).value_counts())
    Unseen_subject_acc_dict[key].append(acc)

In [ ]:
Unseen_subject_acc_dict